In [2]:
import json

def read_jsonl(file):
    # jsonl_data = []
    with open(file, 'r') as f:
        jsonl_data = json.load(f)
    #     for line in f:
    #         jsonl_data.append(line)

    return jsonl_data

props = read_jsonl('../../data/all-wikidata-properties.json')
len(props)

11665

In [3]:
props[0]

{'property': 'http://www.wikidata.org/entity/P6',
 'propertyLabel': 'head of government',
 'propertyDescription': 'head of the executive power of this town, city, municipality, state, country, or other governmental body'}

In [4]:
from sentence_transformers import SentenceTransformer
encoder = SentenceTransformer("all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
from qdrant_client import QdrantClient, models
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance, for testing, CI/CD

In [7]:
qdrant.recreate_collection(
    collection_name="wikidata_properties",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),  # Vector size is defined by used model
        distance=models.Distance.COSINE,
    ),
)

True

In [17]:
out = encoder.encode("Alexei Navalny was born in 1979.")
out.shape

(384,)

In [13]:
# 93 properties without a description
props = [p for p in props if "propertyDescription" in p]

In [20]:
qdrant.upload_records(
    collection_name="wikidata_properties",
    records=[
        models.Record(
            id=idx, vector=encoder.encode(f"{property['propertyLabel']} : {property['propertyDescription']}").tolist(), payload=property
        )
        for idx, property in enumerate(props)
    ],
)

/var/folders/2q/br_c44dn1pq0q31552ypn2800000gn/T/ipykernel_64621/373319611.py:1: DeprecationWarning: `upload_records` is deprecated, use `upload_points` instead
  qdrant.upload_records(


In [21]:
# - in our for loop, encode each sentence and fetch the top five most likely properties from the index 

def fetch_top_n(query:str, collection_name:str, top_n:int) -> dict:
    hits = qdrant.search(
        collection_name,
        query_vector=encoder.encode(query).tolist(),
        limit=top_n,
    )
    return hits

hits = fetch_top_n("Alexei Navalny was born in 1979.", "wikidata_properties", 3)
hits

[ScoredPoint(id=5894, version=0, score=0.43763988209034543, payload={'property': 'http://www.wikidata.org/entity/P6596', 'propertyLabel': 'a-tremasov.ru biography ID', 'propertyDescription': "ID of biographical article at Aleksei Tremasov's project Always with me…"}, vector=None, shard_key=None),
 ScoredPoint(id=6026, version=0, score=0.42259877383642735, payload={'property': 'http://www.wikidata.org/entity/P6737', 'propertyLabel': 'grwar.ru person ID', 'propertyDescription': "ID of biographical article at Alexey Likhotvorik's project Russian Army in the Great War"}, vector=None, shard_key=None),
 ScoredPoint(id=11314, version=0, score=0.4145579052032264, payload={'property': 'http://www.wikidata.org/entity/P12414', 'propertyLabel': 'ru.hayazg.info ID', 'propertyDescription': 'identifier for a person in the Russian edition of the Armenian website hayazg.info'}, vector=None, shard_key=None)]

In [22]:
query = "Alexei Anatolyevich Navalny[c][d] (Russian: Алексей Анатольевич Навальный, IPA: [ɐlʲɪkˈsʲej ɐnɐˈtolʲjɪvʲɪtɕ nɐˈvalʲnɨj]; 4 June 1976 – 16 February 2024) was a Russian opposition leader,[2][4] lawyer, anti-corruption activist, and political prisoner."

hits = fetch_top_n(query, "wikidata_properties", 5)
hits

[ScoredPoint(id=11336, version=0, score=0.49542778270790916, payload={'property': 'http://www.wikidata.org/entity/P12439', 'propertyLabel': 'civil rank', 'propertyDescription': 'non-military rank of a civil office holder in Russian Empire'}, vector=None, shard_key=None),
 ScoredPoint(id=7854, version=0, score=0.46804220658656914, payload={'property': 'http://www.wikidata.org/entity/P8643', 'propertyLabel': 'Dostoyevsky and His Entourage ID', 'propertyDescription': 'identifier for a person somehow connected to Fyodor Dostoyevsky in the website in fedordostoevsky.ru'}, vector=None, shard_key=None),
 ScoredPoint(id=4423, version=0, score=0.46110793557123186, payload={'property': 'http://www.wikidata.org/entity/P5012', 'propertyLabel': 'special rank', 'propertyDescription': 'non-military rank given to a civil office holder in Russia'}, vector=None, shard_key=None),
 ScoredPoint(id=9450, version=0, score=0.45311962712385, payload={'property': 'http://www.wikidata.org/entity/P10462', 'proper